<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/federated_learning_learning_a_simple_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project X
## The objective here is to learn from simple linear model using federated learning techniques
### Requirement : to make a simple model to learn a random dataset and then train them using federated learning , but without using any nn.Modules or Optimizer functions.

In [0]:
pip install syft

In [3]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)
# It overrides pytorch methods and extends pytorch functionality

W0707 12:25:19.118051 139654277666688 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0707 12:25:19.140459 139654277666688 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
bob = sy.VirtualWorker(hook, id = "bob")

Let's create some custom dataset

In [0]:
inputs = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
targets = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [6]:
inputs

(Wrapper)>[PointerTensor | me:40658978729 -> bob:86340691463]

In [7]:
targets

(Wrapper)>[PointerTensor | me:21425771687 -> bob:95673172780]

Let's initialize the weights

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [9]:
weights

(Wrapper)>[PointerTensor | me:15481985579 -> bob:47877597819]

Let's start the training loop which uses mean square error function and using gradient descent step algorithm (here we don;t we have to use any modules or optimizer)

In [10]:
for i in range(10):
  pred = inputs.mm(weights)  # feed forward step
  loss = ((pred - targets)**2).sum()  # calculating our error
  loss.backward()  # taking gradients for error function 
  # in the following lines, we use inplace operations using '_' and do w = w - lr*gradient and so 'sub' is used for subtraction
  weights.data.sub_(weights.grad * 0.1) #  gradient descent step 
  weights.grad *= 0  # initiliazing weights to 0 so that gradients don't get accumulated
  print(loss.get().data)   # printing the losses
  

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


Garbage Collection and Common Errors.
Let's talk about garbage collection.
In pysyft , there is a provision for garbage collection , so that when we delete the pointer created , then it should, simply delete the pointer by saying to worker "hey go and delete the reference from the remote machine".

PySyft beleives that if we created that tensor , then we should be able to control the life cycle of tensor.

In [11]:
bob

<VirtualWorker id:bob #objects:4>

first clear all the objects of bob that it holds

In [12]:
bob = bob.clear_objects()
bob

<VirtualWorker id:bob #objects:0>

In [13]:
bob._objects # now it's all clear 

{}

In [14]:
x = th.tensor([1,2,3,4,5]).send(bob) # creating a torch tensor and sending to bob
bob._objects # let' see bob objects

{15531465098: tensor([1, 2, 3, 4, 5])}

In [0]:
del x  # we are deleting the reference to the pointer 

In [16]:
bob._objects # so now pointer reference is deleted and now bob no longer holds any objects


{}

In [17]:
x = th.tensor([1,2,3,4,5]).send(bob)
x

(Wrapper)>[PointerTensor | me:68401265835 -> bob:84806115412]

In [18]:
bob._objects


{84806115412: tensor([1, 2, 3, 4, 5])}

In [19]:
x.child.garbage_collect_data  # by default garbage collection is set to true, that means when either of pointer is deleted, 
# then worker goes and says to machine to delete the reference pointer 

True

In [0]:
x = "asdf"


In [21]:
bob._objects


{84806115412: tensor([1, 2, 3, 4, 5])}

In following examples we will see that how , in jupyter notebooks , after deleting some pointers , there still some objects created around which never gets garbage collected.
So , when we try to reassign x to a new string , then also bob would not lose reference pointer to the remote machine , and that it is still holding that address as we can see below.

(iT'S NOT ANY ISSUE OF PYSYFT , IT'S JUST SOMETHING THAT JUPYTER DOES)

In [0]:
del x


In [23]:
bob._objects


{84806115412: tensor([1, 2, 3, 4, 5])}

Now , understand this in terms of context of the linear model that we tried to train.

In [0]:
bob = bob.clear_objects()


In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)
    

In [26]:
bob._objects


{3759937253: tensor([1, 2, 3, 4, 5])}

The above example shows that while the for loop works , we are creating so many tensors and sending it to bob , but bob only holds one tensor because everytime loop finishes that tensor is deleted and new tensor is created in place of it through garbage collection, if that were not to happen  then just for simple linear models like above , we would have many thousands of tensors lying around which will take much amount of memory.

COMMON ERRORS 

In [0]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob)
y = th.tensor([1, 2, 3, 4, 5])

In [28]:
z = x + y

TensorsNotCollocatedException: ignored

This is an error due to one of the tensors being on some remote machine named "bob" and the other being present here on the local machine.

In [0]:
alice = sy.VirtualWorker(hook, id = "alice")
x = th.tensor([1, 2, 3, 4, 5]).send(bob)
y = th.tensor([1, 2, 3, 4, 5]).send(alice)

In [31]:
z = x + y

TensorsNotCollocatedException: ignored

This is an error because the addition operation i tried to do on are two different tensors as they are located on two different machines named "bob" and "alice".